# Importing and preparing rental apartments data

## Libraries and settings

In [77]:
# Libraries
import os
import re
import time
import fnmatch
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print(os.getcwd())

/workspaces/DA_Project/Project/Minimum/2&3


## Importing data

In [78]:
# Show .csv - files in the directory
flist = fnmatch.filter(os.listdir('.'), '*.csv')
for i in flist:
    print(i)

# Read the data to a pandas data frame
df = pd.read_csv('comparis_bmw.csv', sep=',', encoding='utf-8')

# Show first records of data frame
df.head()

comparis_bmw.csv
bmw_data_prepared.csv


,web-scraper-order,web-scraper-start-url,text_raw,price_raw,model_raw,type_year_km_transmission_fuel
0,1705078003-1,https://www.comparis.ch/carfinder/marktplatz/b...,BMW X3 X3 xDrive 20d xLine SteptronicCHF 22'90...,CHF 22'900,X3 xDrive 20d xLine Steptronic,Occasion05.2016119.500 kmAutomatDiesel1227 (GE)
1,1705078003-2,https://www.comparis.ch/carfinder/marktplatz/b...,BMW 540 540iCHF 13'750ab CHF 220 / Monat5 / 6O...,CHF 13'750,540i,Occasion05.2006124.900 kmAutomatBenzin4461 (BL)
2,1705078003-3,https://www.comparis.ch/carfinder/marktplatz/b...,BMW 225 Active Tourer 225xe iPerformance Activ...,CHF 28'800,225xe iPerformance Active Tourer Steptronic M ...,Occasion09.201928.950 kmAutomatBenzin/Elektro4...
3,1705078003-4,https://www.comparis.ch/carfinder/marktplatz/b...,BMW 316 316i CompactCHF 1'100Occasion04.200024...,CHF 1'100,316i Compact,Occasion04.2000248.013 kmAutomatBenzin6300 (ZG)
4,1705078003-5,https://www.comparis.ch/carfinder/marktplatz/b...,BMW 530 530xi SteptronicCHF 10'900ab CHF 175 /...,CHF 10'900,530xi Steptronic,Occasion06.2008160.000 kmAutomatBenzin6010 (LU)


## Count number of rows and columns in the data frame

In [79]:
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns
print('Number of columns:', df.shape[1])

Dimension: (990, 6)
Number of rows: 990
Number of columns: 6


## Get data types (raw-format from web scraping)

In [80]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

web-scraper-order                 object
web-scraper-start-url             object
text_raw                          object
price_raw                         object
model_raw                         object
type_year_km_transmission_fuel    object
dtype: object

## Extract and save relevant information from raw data using regular expressions (regex)

### Output type_year_km_transmission_fuel

In [81]:
print(df['type_year_km_transmission_fuel'].head(5), '\n')

0      Occasion05.2016119.500 kmAutomatDiesel1227 (GE)
1      Occasion05.2006124.900 kmAutomatBenzin4461 (BL)
2    Occasion09.201928.950 kmAutomatBenzin/Elektro4...
3      Occasion04.2000248.013 kmAutomatBenzin6300 (ZG)
4      Occasion06.2008160.000 kmAutomatBenzin6010 (LU)
Name: type_year_km_transmission_fuel, dtype: object 



## Extract and save relevant information from raw data using regular expressions (regex)

### Extract Location

In [82]:
# Extrahiere Location
Location = []
for item in df['type_year_km_transmission_fuel']:
    location = re.findall(r'\((.*?)\)', item)
    try:
        Location.append(location[0])
    except:
        Location.append(None)

# Füge Location als neue Spalte hinzu
df['Location'] = pd.Series(Location, dtype='str')

# Zeige die ersten 5 Werte 
print(df['type_year_km_transmission_fuel'].head(5), '\n')
print(df['Location'].head(5))

0      Occasion05.2016119.500 kmAutomatDiesel1227 (GE)
1      Occasion05.2006124.900 kmAutomatBenzin4461 (BL)
2    Occasion09.201928.950 kmAutomatBenzin/Elektro4...
3      Occasion04.2000248.013 kmAutomatBenzin6300 (ZG)
4      Occasion06.2008160.000 kmAutomatBenzin6010 (LU)
Name: type_year_km_transmission_fuel, dtype: object 

0    GE
1    BL
2    BL
3    ZG
4    LU
Name: Location, dtype: object


### Extract PLZ

In [83]:
# Extrahiere PLZ
PLZ = []
for item in df['type_year_km_transmission_fuel']:
    plz = re.search(r'(\d{4}) \(([^)]+)\)', item)
    if plz:
        PLZ.append(plz.group(1))
    else:
        PLZ.append(None)

# Füge PLZ als neue Spalte hinzu
df['PLZ'] = pd.Series(PLZ, dtype="Int64")

# Zeige die ersten 5 Werte
print(df['type_year_km_transmission_fuel'].head(5), '\n')
print(df['PLZ'].head(5))

0      Occasion05.2016119.500 kmAutomatDiesel1227 (GE)
1      Occasion05.2006124.900 kmAutomatBenzin4461 (BL)
2    Occasion09.201928.950 kmAutomatBenzin/Elektro4...
3      Occasion04.2000248.013 kmAutomatBenzin6300 (ZG)
4      Occasion06.2008160.000 kmAutomatBenzin6010 (LU)
Name: type_year_km_transmission_fuel, dtype: object 

0    1227
1    4461
2    4460
3    6300
4    6010
Name: PLZ, dtype: Int64


### Extract km

In [84]:
# Extrahiere km
KM = []
for item in df['type_year_km_transmission_fuel']:
    km_raw = re.search(r'.\d{4}(\d+\.\d+)', item)
    if km_raw:
        KM.append(km_raw.group(1))
    else:
        KM.append(None)

# Füge Year als neue Spalte hinzu
df['km_raw'] = pd.Series(KM, dtype="object")

# Zeige die ersten 5 Werte
print(df['type_year_km_transmission_fuel'].head(5), '\n')
print(df['km_raw'].head(5))

0      Occasion05.2016119.500 kmAutomatDiesel1227 (GE)
1      Occasion05.2006124.900 kmAutomatBenzin4461 (BL)
2    Occasion09.201928.950 kmAutomatBenzin/Elektro4...
3      Occasion04.2000248.013 kmAutomatBenzin6300 (ZG)
4      Occasion06.2008160.000 kmAutomatBenzin6010 (LU)
Name: type_year_km_transmission_fuel, dtype: object 

0    119.500
1    124.900
2     28.950
3    248.013
4    160.000
Name: km_raw, dtype: object


### Extract year

In [85]:
# Extrahiere year
YEAR = []
for item in df['type_year_km_transmission_fuel']:
    year = re.search(r'.(\d{4})', item)
    if year:
        YEAR.append(year.group(1))
    else:
        YEAR.append(None)

# Füge Year als neue Spalte hinzu
df['Year'] = pd.Series(YEAR, dtype="Int64")

# Zeige die ersten 5 Werte
print(df['type_year_km_transmission_fuel'].head(5), '\n')
print(df['Year'].head(5))

0      Occasion05.2016119.500 kmAutomatDiesel1227 (GE)
1      Occasion05.2006124.900 kmAutomatBenzin4461 (BL)
2    Occasion09.201928.950 kmAutomatBenzin/Elektro4...
3      Occasion04.2000248.013 kmAutomatBenzin6300 (ZG)
4      Occasion06.2008160.000 kmAutomatBenzin6010 (LU)
Name: type_year_km_transmission_fuel, dtype: object 

0    2016
1    2006
2    2019
3    2000
4    2008
Name: Year, dtype: Int64


### Extraction of all Information

In [86]:
# Extrahieren von Typ, Jahr, Kilometerstand, Getriebeart, Treibstofftyp, Postleitzahl und Location
extracted = df['type_year_km_transmission_fuel'].str.extract(r'(\w+)(\d{2}.\d{4})(\d+\.\d+ km)([A-Z]\w+)([A-Z]\w+)(\d{4}) \(([^)]+)\)')
extracted.columns = ['Type', 'Year', 'Kilometers', 'Transmission', 'Fuel', 'PLZ', 'Location']

# Ausgabe der ersten 5 extrahierten Datensätze
print(extracted.head())

       Type     Year  Kilometers Transmission    Fuel   PLZ Location
0  Occasion  05.2016  119.500 km      Automat  Diesel  1227       GE
1  Occasion  05.2006  124.900 km      Automat  Benzin  4461       BL
2       NaN      NaN         NaN          NaN     NaN   NaN      NaN
3  Occasion  04.2000  248.013 km      Automat  Benzin  6300       ZG
4  Occasion  06.2008  160.000 km      Automat  Benzin  6010       LU


### Output price_raw

In [87]:
print(df['price_raw'].head(5), '\n')

0    CHF 22'900
1    CHF 13'750
2    CHF 28'800
3     CHF 1'100
4    CHF 10'900
Name: price_raw, dtype: object 



In [88]:
# Verwende str.extract(), um Zahlen aus der 'price_raw'-Spalte zu extrahieren
df['price'] = df['price_raw'].str.extract(r"(\d+['\d+]*)", expand=False)
df['price'] = df['price'].str.replace("'", "").astype('Int64')  # Entferne Apostrophe und ändere den Datentyp in 'Int64'

# Gib die ersten 5 Werte der 'price_raw'- und 'price'-Spalten aus
print(df['price_raw'].head(5), '\n')
print(df['price'].head(5))

0    CHF 22'900
1    CHF 13'750
2    CHF 28'800
3     CHF 1'100
4    CHF 10'900
Name: price_raw, dtype: object 

0    22900
1    13750
2    28800
3     1100
4    10900
Name: price, dtype: Int64


### Output km_raw

In [89]:
print(df['km_raw'].head(5), '\n')

0    119.500
1    124.900
2     28.950
3    248.013
4    160.000
Name: km_raw, dtype: object 



In [90]:
# Verwende str.extract(), um Zahlen aus der 'km'-Spalte zu extrahieren
df['km'] = df['km_raw'].str.extract(r"(\d+\.\d+)", expand=False)
df['km'] = df['km_raw'].str.replace(".", "").astype('Int64')  # Entferne Apostrophe und ändere den Datentyp in 'Int64'

# Gib die ersten 5 Werte der 'km'- und 'km'-Spalten aus
print(df['km_raw'].head(5), '\n')
print(df['km'].head(5))

0    119.500
1    124.900
2     28.950
3    248.013
4    160.000
Name: km_raw, dtype: object 

0    119500
1    124900
2     28950
3    248013
4    160000
Name: km, dtype: Int64


### Get data types of all variables including the new ones

In [91]:
df.dtypes

web-scraper-order                 object
web-scraper-start-url             object
text_raw                          object
price_raw                         object
model_raw                         object
type_year_km_transmission_fuel    object
Location                          object
PLZ                                Int64
km_raw                            object
Year                               Int64
price                              Int64
km                                 Int64
dtype: object

## Create additional variables

### Change string 'text_raw' to uppercase 

In [92]:
# Change strings to uppercase

df['text_raw'] = df['text_raw'].str.upper()
print(df['text_raw'].head(10))

0    BMW X3 X3 XDRIVE 20D XLINE STEPTRONICCHF 22'90...
1    BMW 540 540ICHF 13'750AB CHF 220 / MONAT5 / 6O...
2    BMW 225 ACTIVE TOURER 225XE IPERFORMANCE ACTIV...
3    BMW 316 316I COMPACTCHF 1'100OCCASION04.200024...
4    BMW 530 530XI STEPTRONICCHF 10'900AB CHF 175 /...
5    BMW X5 X5 XDRIVE 40D STEPTRONICCHF 33'900AB CH...
6    BMW 525 525D TOURINGCHF 3'3005.5 / 6OCCASION10...
7    BMW 225 ACTIVE TOURER 225I XDRIVE ACTIVE TOURE...
8    BMW 435 435I COUPÉ XDRIVE SPORT LINE STEPTRONI...
9    BMW X5 X5 XDRIVE 48V 40D M SPORT PRO STEPTRONI...
Name: text_raw, dtype: object


In [93]:
# Show first item of variable 'text_raw'
print(df['text_raw'][0])

BMW X3 X3 XDRIVE 20D XLINE STEPTRONICCHF 22'900AB CHF 367 / MONAT4 / 6OCCASION05.2016119.500 KMAUTOMATDIESEL1227 (GE)PREISCHF 22'900FINANZIERUNGAB CHF 367 / MONAT4 / 6ANFRAGEN MERKENDETAILS


### Create new binary variables (0/1) luxury, competition, xDrive, automat

In [94]:
# Create patterns which can be used to search the variable 'text_raw'
patternLuxury = '(LUXURY)'
patternCompetition = '(COMPETITION)'
patternXdrive = '(XDRIVE)'
patternAutomat = '(AUTOMAT)'

# Create new variables as binary dummy (0/1) variable
# text_raw beinhaltet mehr Informationen
df['luxury'] = df['text_raw'].str.contains(pat = patternLuxury).astype(int)
df['competition'] = df['text_raw'].str.contains(pat = patternCompetition).astype(int)
df['xDrive'] = df['text_raw'].str.contains(pat = patternXdrive).astype(int)
df['automat'] = df['text_raw'].str.contains(pat = patternAutomat).astype(int)

print(df['luxury'].sum())
print(df['competition'].sum())
print(df['xDrive'].sum())
print(df['automat'].sum())

# Show values
df[['text_raw','luxury', 'competition','xDrive','automat']]

23
52
662
883


,text_raw,luxury,competition,xDrive,automat
0,BMW X3 X3 XDRIVE 20D XLINE STEPTRONICCHF 22'90...,0,0,1,1
1,BMW 540 540ICHF 13'750AB CHF 220 / MONAT5 / 6O...,0,0,0,1
2,BMW 225 ACTIVE TOURER 225XE IPERFORMANCE ACTIV...,0,0,0,1
3,BMW 316 316I COMPACTCHF 1'100OCCASION04.200024...,0,0,0,1
4,BMW 530 530XI STEPTRONICCHF 10'900AB CHF 175 /...,0,0,0,1
...,...,...,...,...,...
985,BMW 520 520D TOURING STEPTRONICCHF 12'000AB CH...,0,0,0,1
986,BMW 535 535I XDRIVE TOURING STEPTRONICCHF 12'5...,0,0,1,1
987,BMW 330 330I XDRIVE STEPTRONIC M SPORTCHF 42'8...,0,0,1,1
988,BMW X3 X3 XDRIVE 30D STEPTRONICCHF 11'600AB CH...,0,0,1,1


### Create new categorical variable based on km

In [95]:
# Define classes (labels)
labels = ['0-50k', '50-120k', '120k+']

# Use the .cut method from pandas to divide the numeric values in classes
df["km_cat"] = pd.cut(df['km'], bins=[0, 50000, 120000, 500000], labels=labels)

# Show original data and classes
df[['km', 'km_cat']].head(10)

,km,km_cat
0,119500,50-120k
1,124900,120k+
2,28950,0-50k
3,248013,120k+
4,160000,120k+
5,125900,120k+
6,336000,120k+
7,97000,50-120k
8,112500,50-120k
9,<NA>,NaN


### Create new numeric variable 'km_per_year'

In [96]:
# Create the new variable
df['km_per_year'] = round(df['km'] / (2024-df['Year']), 2)

# Show values
df[['Year','km','price','km_per_year']].head(10)

,Year,km,price,km_per_year
0,2016,119500,22900,14937.5
1,2006,124900,13750,6938.89
2,2019,28950,28800,5790.0
3,2000,248013,1100,10333.88
4,2008,160000,10900,10000.0
5,2015,125900,33900,13988.89
6,2004,336000,3300,16800.0
7,2017,97000,13500,13857.14
8,2014,112500,22950,11250.0
9,2024,<NA>,112300,<NA>


### Create new categorical variable based on km

In [97]:
# Define classes (labels)
labels = ['low', 'medium', 'high']

# Use the .cut method from pandas to divide the numeric values in classes
df["km_per_year_cat"] = pd.cut(df['km_per_year'], bins=[0, 10000, 15000, 50000], labels=labels)

# Show original data and classes
df[['km_per_year','km_cat','km_per_year_cat']].head(10)

,km_per_year,km_cat,km_per_year_cat
0,14937.5,50-120k,medium
1,6938.89,120k+,low
2,5790.0,0-50k,low
3,10333.88,120k+,medium
4,10000.0,120k+,low
5,13988.89,120k+,medium
6,16800.0,120k+,high
7,13857.14,50-120k,medium
8,11250.0,50-120k,medium
9,<NA>,NaN,NaN


## Count and identify missing values (if any)

In [98]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values
df[df.isna().any(axis=1)].head()

web-scraper-order                   0
web-scraper-start-url               0
text_raw                            0
price_raw                           0
model_raw                           0
type_year_km_transmission_fuel      0
Location                            0
PLZ                                 0
km_raw                            113
Year                                0
price                               0
km                                113
luxury                              0
competition                         0
xDrive                              0
automat                             0
km_cat                            113
km_per_year                       113
km_per_year_cat                   118
dtype: int64


,web-scraper-order,web-scraper-start-url,text_raw,price_raw,model_raw,type_year_km_transmission_fuel,Location,PLZ,km_raw,Year,price,km,luxury,competition,xDrive,automat,km_cat,km_per_year,km_per_year_cat
9,1705078003-10,https://www.comparis.ch/carfinder/marktplatz/b...,BMW X5 X5 XDRIVE 48V 40D M SPORT PRO STEPTRONI...,CHF 112'300,X5 xDrive 48V 40d M Sport Pro Steptronic,Occasion01.202410 kmAutomatElektro/Diesel1530 ...,VD,1530,None,2024,112300,<NA>,0,0,1,1,NaN,<NA>,NaN
11,1705078007-12,https://www.comparis.ch/carfinder/marktplatz/b...,BMW X3 X3 XDRIVE 30ECHF 86'999AB CHF 1'393 / M...,CHF 86'999,X3 xDrive 30e,Neuwagen10 kmAutomatBenzin/Elektro4147 (BL),BL,4147,None,4147,86999,<NA>,0,0,1,1,NaN,<NA>,NaN
12,1705078007-13,https://www.comparis.ch/carfinder/marktplatz/b...,BMW X1 X1 XDRIVE 30ECHF 69'900AB CHF 1'119 / M...,CHF 69'900,X1 xDrive 30e,Neuwagen10 kmAutomatBenzin/Elektro3048 (BE),BE,3048,None,3048,69900,<NA>,0,0,1,1,NaN,<NA>,NaN
14,1705078007-15,https://www.comparis.ch/carfinder/marktplatz/b...,BMW IX IX1 XDRIVE 30 M SPORTCHF 71'900AB CHF 1...,CHF 71'900,iX1 xDrive 30 M Sport,Neuwagen10 kmAutomatElektro8048 (ZH),ZH,8048,None,8048,71900,<NA>,0,0,1,1,NaN,<NA>,NaN
15,1705078007-16,https://www.comparis.ch/carfinder/marktplatz/b...,BMW IX IX XDRIVE40CHF 99'900AB CHF 1'599 / MON...,CHF 99'900,iX xDrive40,Neuwagen10 kmAutomatElektro9500 (SG),SG,9500,None,9500,99900,<NA>,0,0,1,1,NaN,<NA>,NaN


## Count and identify duplicated values (if any)

It seems like there are no duplicates in the DataFrame

In [99]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['web-scraper-order', 'text_raw', 'model_raw']].duplicated()]

0


,web-scraper-order,web-scraper-start-url,text_raw,price_raw,model_raw,type_year_km_transmission_fuel,Location,PLZ,km_raw,Year,price,km,luxury,competition,xDrive,automat,km_cat,km_per_year,km_per_year_cat


### Save data to file

In [100]:
df.to_csv('bmw_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### SQLLite3

In [101]:
import sqlite3
 
# Daten laden, indem Sie den vollständigen Pfad zur Datei angeben
bereinigte_daten = pd.read_csv('/workspaces/DA_Project/Project/Minimum/2&3/bmw_data_prepared.csv')
 
# Verbindung zur Datenbank herstellen
conn = sqlite3.connect('bmw_datenbank.db')
 
# Daten in die Datenbank einfügen
bereinigte_daten.to_sql('bmw_tabelle', conn, index=False, if_exists='replace')
 
# Verbindung schließen
conn.close()

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [102]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.2.0-1018-azure
Datetime: 2024-01-13 12:17:28
Python Version: 3.10.13
-----------------------------------
